### Dedicated for extracting faces from any given input using the finetuned Detectron2 model
(Either a video, a multi-faced image or the custom dataset used to evaluate the system)

#### Installing Detectron2

In [0]:
!pip install -q cython pyyaml==5.1
!pip install -q -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -q -e detectron2_repo
!gdown --id 18Ev2bpdKsBaDufhVKf0cT6RmM3FjW3nL

#### Imports

In [0]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2 import model_zoo
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
import time

#### Load the finetuned detectron2
Specify hyperparameters to run the model

In [0]:
cfg = get_cfg()
cfg.DATASETS.TRAIN = ("faces_train",)
cfg.DATASETS.TEST = ("faces_val", )
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.merge_from_file(
  model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
  )
)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
statement_metadata = MetadataCatalog.get("faces_train")
cfg.MODEL.WEIGHTS = "face_detector.pth"
predictor = DefaultPredictor(cfg)

#### Detect faces in a image

In [0]:
def detect_img(im, mode='single'):
  ''' Finds the bounding boxes of faces in an image '''
  
  outputs = predictor(im)
  v = Visualizer(
    im[:, :, ::-1],
    metadata=statement_metadata, 
    scale=1., 
    instance_mode=ColorMode.IMAGE
  )
  instances = outputs["instances"].to("cpu")
  instances.remove('pred_masks')
  # If multiple faces, save in numpy array otherwise in a csv
  if mode == 'multiple':
    bounding_box = np.array(instances.get('pred_boxes').tensor).astype('int')
    return bounding_box
  else:
    if np.array(instances.get('pred_boxes').tensor).shape[0] == 0:
      plt.imshow(im)
      return -1, -1, -1, -1, None
    else:
      bounding_box = np.array(instances.get('pred_boxes').tensor)[0].astype('int')
      x1, y1, x2, y2 = bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3]
  
  # Prepare an image that contains the face bounds
  v = v.draw_instance_predictions(instances)
  result = v.get_image()[:, :, ::-1]
  return x1, y1, x2, y2, result

#### Detect faces in a multi-faced image

In [0]:
def get_multiple_boxes(folder_path):
  ''' If image known to have multiple faces, save them in a Numpy array '''

  images = []
  boxes = []
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isdir(file_path):
      continue
    
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    box = detect_img(img, mode='multiple')
    images.append(file_path)
    boxes.append(box)
    
  np.save(folder_path+'/arrays/file_names.npy', np.array(images))
  np.save(folder_path+'/arrays/boxes.npy', np.array(boxes))

all_path = '/content/drive/My Drive/ML/Face_dataset/all_faces'
get_multiple_boxes(all_path)

#### Save detected faces in a Dataframe for recognition purposes

In [0]:
def get_boxes(dataset_path):
  ''' If the images known to have single faces, 
  save them in a csv file for training the SVM 
  '''

  images = []
  dataset = []
  # Enumerate all images for all identity classes
  for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      img = cv2.imread(file_path)
      x1, y1, x2, y2, result = detect_img(img)
      # Save the bounding box and the face label
      data = {
          'file_name': file_path,
          'x_min': x1, 'x_max': x2,
          'y_min': y1, 'y_max': y2,
          'label': folder
      }
      dataset.append(data)
  return dataset

# Save the training and testing data in CSV files
train_path = '/content/drive/My Drive/ML/Face_dataset/Train/'
train_dataset = get_boxes(train_path)
df_train = pd.DataFrame(train_dataset)

test_path = '/content/drive/My Drive/ML/Face_dataset/Test/'
test_dataset = get_boxes(test_path)
df_test = pd.DataFrame(test_dataset)

df_train.to_csv('/content/drive/My Drive/ML/Face_dataset/train_boxes.csv')
df_test.to_csv('/content/drive/My Drive/ML/Face_dataset/test_boxes.csv')

#### Test Face detection on a personal video

In [0]:
def detect_frames(path):
  ''' Detect faces in an input video and save them in a CSV file '''

  vidcap = cv2.VideoCapture(path)
  # Video parameters
  width  = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
  height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  timer = []
  video_faces = []
  # Output video
  fourcc = cv2.VideoWriter_fourcc(*"XVID")
  video = cv2.VideoWriter('drive/My Drive/ML/detected_faces.avi', fourcc, fps, (height, width))
  # Read frames
  while True:
    ret, image = vidcap.read()
    if not ret:
      break
    image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    t0 = time.time()
    x1, y1, x2, y2, result = detect_img(image)
    timer.append(time.time() - t0)
    data = {
          'x_min': x1, 'x_max': x2,
          'y_min': y1, 'y_max': y2,
    }
    video_faces.append(data)
    # if frame contains a face, save the face otherwise save the original frame
    if result is None:
      video.write(image)
    else:
      video.write(result)
  # Save the detected faces in a CSV file
  df_video = pd.DataFrame(video_faces)
  df_video.to_csv('/content/drive/My Drive/ML/Face_dataset/video_boxes.csv')
  cv2.destroyAllWindows()
  video.release()
  print('Average detection time for 1 frame', np.average(np.array(timer)))

video_path = 'drive/My Drive/ML/IMG_5947.MOV'
detect_frames(video_path)